In [1]:
import os
os.chdir("/home/emma/Documents/NLP/Topic Modeling sur Grand Debat")

import pandas as pd
import geopandas as gpd
import numpy as np

## Correspondance Code Postal - Arrondissement

In [ ]:
# On utilise 2 bases car aucune des 2 n'est vraiment complète
# Base communes - codes postaux 1 :
# + : contient plus de codes postaux de la métropole
# + : indique l'arrondissement
# - : ne contient pas les domtom
zc_df = pd.read_csv("Data/GeoData/code-insee-postaux-geoflar.csv", sep=";", dtype=str)

zc_df.columns = [ name.replace(" ", "_").lower() for name in zc_df.columns]
zc_df= zc_df[["code_insee","code_postal","nom_dept","code_dept","nom_region","code_arrondissement"]] 

# Base communes - codes postaux 2
# - : ne donne pas l'arrondissement
# + : contient les domtom
zc_df2 = pd.read_csv("Data/GeoData/laposte_hexasmal.csv", sep=";",dtype=str)
zc_df2 = zc_df2.rename(columns={"Code_commune_INSEE":"code_insee", "Code_postal":"code_postal"})
zc_df2 = zc_df2[["code_insee","code_postal"]]


# Affectation commune - arrondissent
city_df = pd.read_csv("Data/GeoData/liste-communes.csv", sep=",", dtype=str, encoding="latin-1")
city_df.columns = [name.lower() for name in city_df.columns]

In [ ]:
# Créer code arrondissement à partir du code départment et du numéro d'arrondissement
# Attention cas spécial pour les domotom
zc_df["code_arr"] = np.where(zc_df.code_dept.str.len()==2, zc_df.code_dept + "00" + zc_df.code_arrondissement, zc_df.code_dept + "0" + zc_df.code_arrondissement) 
city_df["code_arr"] = np.where(city_df.dep.str.len()==2, city_df.dep + "00" + city_df.ar, city_df.dep + "0" + city_df.ar) 

# Code Insee : code département + code commune
city_df["code_insee"] = city_df.dep + "" + city_df.com

city_df.head()
zc_df.head()
zc_df.head()

In [ ]:
# Rajoute codes postaux sur la table de correspondance commune-arrondissement
city_df  = city_df.merge(zc_df2, on="code_insee", how="inner")

# Merge la table correspondance commune-arrondissement
zc_df = zc_df.merge(city_df[["code_insee", "code_arr", "code_postal"]], on="code_insee", how="outer", suffixes=("","_b"))
zc_df.head()

In [ ]:
# 3 cas :
# 1. Arrondissement était manquant dans la base code postal 1 => on prend celui de la table correspondance
zc_df.code_arr = zc_df.code_arr.fillna(zc_df.code_arr_b)
# 2. Conflit entre la base Codes Postaux 1 et la table de correspondace au niveau de l'arrondissement 
# => on prend l'arrondissment de la table de correspondance car cette table est plus récente
# (il y a eu des changements en 2017)
zc_df.code_arr = np.where(((zc_df.code_arr != zc_df.code_arr_b)&(~ zc_df.code_arr_b.isnull())),
                          zc_df.code_arr_b, 
                          zc_df.code_arr)
# 3. Arrondissement manquant dans les 2 bases : on supprime ces lignes
zc_df = zc_df.dropna(subset=["code_arr"])

In [ ]:
# Comme un code postal = plusieurs communes, il arrive parfois que certaines villes partagent le même code
# postal mais pas le même arrondissement => on ne peut pas faire de correspondance parfaite entre code postal et
# arrondissement.

# Malgrè tout, on choisit d'attribuer à un code postal, l'arrondissement départemental le plus courant parmi les 
# les villes qui partagent ce code postal
# Exemple :
# Ville1,Ville2, Ville3 et Ville4 ont le même code postal XXXXX
# Ville1 Arr=1; Ville2 Arr=1; Ville3 Arr=2; Ville4 Arr=1 
# Au code postal XXXXX, on  associe l'arrondissement => 1

top_arr = zc_df[["code_postal", "code_arr"]].groupby("code_postal").agg(lambda x: x.value_counts().index[0])\
                                            .reset_index()
top_arr.head()

In [ ]:
#Export
top_arr.to_csv("Data/GeoData/corresp_arrondissement_code_postal.csv", index=False)

## Rajouter noms du département et de la région dans le GEOJSON avec les arrondissement départementaux

In [ ]:
# GeoJSON avec contours des arrondissements départementaux
geo_df = gpd.read_file("Data/GeoData/arrondissements-avec-outre-mer.geojson")
geo_df_44 = gpd.read_file("Data/GeoData/arrondissements44.geojson")
 
# Base avec les noms des départements et de leur région
dep_df = pd.read_csv("Data/GeoData/departements-france.csv", sep=",",dtype=str)
dep_df = dep_df.rename(columns= {"code_departement":"code_dep", "nom_departement":"nom_dep", "nom_region":"nom_region"})
dep_df.head()

In [ ]:
geo_df = geo_df.append(geo_df_44[(geo_df_44.code == "44001")|(geo_df_44.code == "44004")],ignore_index=True)

In [ ]:
# Extraire le code du département depuis le code arrondissement
geo_df["code_dep"] = geo_df.code.str[:2]
geo_df["code_dep"] =  np.where((geo_df["code_dep"]=="97")| (geo_df["code_dep"]=="98"),
                               geo_df.code.str[:3],
                               geo_df.code_dep)
geo_df.head()

In [ ]:
geo_df_44

In [ ]:
geo_df[geo_df.code_dep=="44"]

In [ ]:
geo_df = geo_df.merge(dep_df[["code_dep", "nom_dep", "nom_region"]], on="code_dep", how="left")
geo_df.head()

In [ ]:
# Export
geo_df.to_file("Data/GeoData/arrondissements-dep-region.geojson", driver='GeoJSON')


## Rajouter les noms des régions dans le GeoJson avec les départements

In [10]:
geo_df = gpd.read_file("Data/GeoData/departements-avec-outre-mer.geojson")
geo_df.head()

# Base avec les noms des départements et de leur région
dep_df = pd.read_csv("Data/GeoData/departements-france.csv", sep=",",dtype=str)
dep_df = dep_df.rename(columns= {"code_departement":"code_dep", "nom_departement":"nom_dep", "nom_region":"nom_region"})
dep_df.head()

,code_dep,nom_dep,code_region,nom_region
0,01,Ain,84,Auvergne-Rhône-Alpes
1,02,Aisne,32,Hauts-de-France
2,03,Allier,84,Auvergne-Rhône-Alpes
3,04,Alpes-de-Haute-Provence,93,Provence-Alpes-Côte d'Azur
4,05,Hautes-Alpes,93,Provence-Alpes-Côte d'Azur


In [11]:
geo_df.head()


,code,nom,geometry
0,971,Guadeloupe,"(POLYGON ((-61.79038 16.16682, -61.79164 16.16..."
1,972,Martinique,"(POLYGON ((-60.86733 14.38867, -60.86802 14.38..."
2,973,Guyane,"(POLYGON ((-53.87049 5.74494, -53.86464 5.7429..."
3,976,Mayotte,"(POLYGON ((45.04101 -12.64686, 45.04139 -12.64..."
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ..."


In [12]:
geo_df = geo_df.merge(dep_df[["code_dep", "nom_region"]], left_on="code", right_on="code_dep", how="left")
geo_df.head()

,code,nom,geometry,code_dep,nom_region
0,971,Guadeloupe,"(POLYGON ((-61.79038 16.16682, -61.79164 16.16...",971,Guadeloupe
1,972,Martinique,"(POLYGON ((-60.86733 14.38867, -60.86802 14.38...",972,Martinique
2,973,Guyane,"(POLYGON ((-53.87049 5.74494, -53.86464 5.7429...",973,Guyane
3,976,Mayotte,"(POLYGON ((45.04101 -12.64686, 45.04139 -12.64...",976,Mayotte
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",01,Auvergne-Rhône-Alpes


In [13]:
geo_df = geo_df.drop(["code_dep"], axis=1)

In [15]:
# Export
geo_df.to_file("Data/GeoData/departements-avec-regions.geojson", driver='GeoJSON')
